In [2]:
import pandas as pd
import glob
import os

# Convert all data from JSON to a single CSV and Excel file

In [ ]:
files_jatim = glob.glob('../data/dapo-kemdikdasmen/Jawa-Timur/*/*/*.json')

temp_df_jatim = []

for f in files_jatim:
    df = pd.read_json(f)
    filename = os.path.splitext(os.path.basename(f))[0]
    df.insert(0, 'sekolah', filename)
    temp_df_jatim.append(df)

df_jatim = pd.concat(temp_df_jatim, ignore_index=True)

df_jatim.to_csv('../data/dapo-kemdikdasmen/Jawa-Timur/ALL/jatim-csv.csv', index=False)
df_jatim.to_excel('../data/dapo-kemdikdasmen/Jawa-Timur/ALL/jatim-excel.xlsx', index=False)